# Gaussian Filters Prelude
As noted in the previous chapter's [notes](../02_RecursiveStateEst/2_BayesFilterIntro.ipynb), each family of Bayesian Filters have characteristics which make them advantageous at times, but at the cost of introducing some bias and accepting other limitations. Here, we'll explore *Gaussian Filters* -- these are filters which assume that each input dimension is represented by a Gaussian (Normal) Distribution. As a brief reminder, this means that for the probability distribution of a random vector, $X$, is represented as follows:

$$
p(X) = \frac{1}{\sqrt{(2\pi)^n \left| \Sigma \right|}} \exp \left({-\frac{1}{2} (X-\mu)^\top \Sigma^{-1} (X-\mu)} \right) \: \text{where} \: X \in \mathbb{R}^n
$$

This carries a few consequences:
* Each dimension is continuous and infinite
* Each density is unimodal (that is, there is a single peak in the distribution)
* Each density is symmetric
* The *mean* and *covariance* are the ONLY moments of the distribution

# Origins and Criteria of the Kalman Filter
This technique was invented by Swerling (1958) and Kalman (1960) to both filter and create predictions of *linear systems* with Gaussian errors. In essence, for each moment in time, belief (posterior) is represented by both mean $\mu_t$ and covariance $\Sigma_t$. For this to be valid, three conditions must be met:

**1. State Transition $p(x_t \: | \: u_t, x_{t-1})$ must be a linear transformation.**
Particularly, even though we're treating this as a conditional probability, the reality is this implies that we can extrapolate $x_{t-1}$ to $x_t$ with noise $\epsilon_t$. This implies that a *state model* is known *a priori*. This is best summarized as:

$$
x_t = Ax_{t-1} + Cu_t + \epsilon_t
$$

Note that in this case, $C$ is the *control transformation* of control signal $u_t$. That means that $u \in \mathbb{R}^m$; it is not bound to the same dimensional space as $x \in \mathbb{R}^n$. Therefore, to maintain linear consistency, the dimensions of $C$ must be $n \times m$.

Remember that the system noise parameter $\epsilon_t$ is *Normally Distributed*. That is to say $\epsilon_t \sim N(0, R_t) \in \mathbb{R}^n$.

**2. The Measurement Distribution $p(z_t \: | \: x_t)$ must also be a linear transformation.**
Like before, we're expecting that sensor measurements can be derived from the state with noise. Note, sometimes sensors cannot perfectly tell us every dimension of the state vector we're estimating (for example, an IMU cannot give exact geographic coordinates, but it can tell you velocity and higher-order differential vectors). This means that the sensor data vector $z_t \in \mathbb{R}^k$. We'll summarize this model as:

$$
z_t = H_t x_t + \delta_t
$$

Just as noted with the system noise, $\delta_t$ is also *Normally Distributed*: $\delta_t \sim N(0, Q_t) \in \mathbb{R}^k$.

**3. The Initial (*a priori*) Belief must be Gaussian Distributed.**
While this seems obvious, it might be difficult to model. Sometimes we have perfect knowledge of the initial state (such as when the system starts, we're at known coordinates), and sometimes there's very little certainty of a distribution (for instance, we're told by another system \[or person\] information before the phenomenon has crossed the sensor horizon). The confusing part is leveraging exceptionally small or large covariances with the initial.


## The Kalman System Model
Loosely, the system model can be described with the following diagram:

![Kalman System](images/KalmanSys.png)

# The Kalman Filter Algorithm
Again, this is a *type* of Bayesian Filter, so it has two phases of calculation: Prediction & Correction.


## Pseudocode Algorithm
**KalmanFilter**($\mu_{t-1}, \Sigma_{t-1}, u_t, z_t$):
- Predict $\mu_t, \Sigma_t$
  - $\overline{\mu_t} = A_t \mu_{t-1} + C_t u_t$
  - $\overline{\Sigma_t} = A_t \Sigma_{t-1} A_t^\top + R_t$
- Correct estimate of $\mu_t, \Sigma_t$
  - $K_t = \overline{\Sigma_t} H_t^\top (H_t \overline{\Sigma_t} H_t^\top + Q_t)^{-1}$
  - $\mu_t = \overline{\mu_t} + K_t (z_t - H_t \overline{\mu_t})$
  - $\Sigma_t = (I - K_t H_t) \overline{\Sigma_t}$
- return $\mu_t$, $\Sigma_t$


## The Algorithm Explained
First, let's consider the expanded probability distributions of $p(x_t \: | \: u_t, x_{t-1})$ and $p(z_t \: | \: x_t)$...

$$
p(x_t \: | \: u_t, x_{t-1}) = \frac{1}{\sqrt{(2\pi)^n \mid R_t \mid}} \cdot \exp \left({ \frac{-1}{2} (x_t - A_t x_{t-1} - B_t u_t)^\top R_t^{-1} (x_t - A_t x_{t-1} - B_t u_t) }\right) \\
p(z_t \: | \: x_t) = \frac{1}{\sqrt{(2\pi)^k \mid Q_t \mid}} \cdot \exp \left({ \frac{-1}{2} (z_t - C_t x_t)^\top Q_t^{-1} (z_t - C_t x_t) }\right)
$$

These expansions, though complicated in appearance, are a logical conclusion from the distributions of both $\epsilon_t$ and $\delta_t$ -- Adding a constant to a normally distributed random variable only moves the mean and has no effect on the covariance. With regards to the system noise, this makes perfect sense: we can't control higher-order effects which appear as noise in our system. For our sensor, this embeds an assumption: the sensor itself has NO systemic noise or bias (as these force the error distribution to be non-Gaussian).